In [1]:
# constants
import math

width = 256
chunks = 6
digits = 52
startdenom = int(math.pow(width, chunks))
significance = int(math.pow(2, digits))
overflow = significance * 2
mask = width - 1

In [2]:
# from pycasso.constants import width, mask
class ARC4:
    def __init__(self, key):
        self.key = key

        global keylen, t, i, j, s
        keylen = len(self.key)
        t = None
        i = 0
        j = self.i = self.j = 0
        s = self.S = []

    def main(self):
        global i, j

        while i < width:
            s.append(i)
            i += 1

        for i in range(width):
            t = s[i]
            j = mask & (j + self.key[i % keylen] + t)
            s[i] = s[j]
            s[j] = t

        self.g(width)

    def g(self, count):
        t = None
        r = 0
        i = self.i
        j = self.j
        s = self.S

        while count:
            i = mask & (i + 1)
            t = s[i]
            j = mask & (j + t)
            s[i] = s[j]
            s[j] = t
            r = r * width + s[mask & (s[i] + s[j])]
            count -= 1

        self.i = i
        self.j = j

        return r

In [3]:
from time import time

#from pycasso.constants import (
#    width,
#    chunks,
#    startdenom,
#    significance,
#    overflow,
#    mask
#)

# from pycasso.cipher import ARC4


def seedrandom(seed):
    key = []
    #print(seed)
    shortseed = mixkey(seed, key)
    #print(shortseed)
    arc4 = ARC4(shortseed)
    arc4.main()

    def prng():
        n = arc4.g(chunks)
        d = startdenom
        x = 0

        while n < significance:
            n = (n + x) * width
            d *= width
            x = arc4.g(1)

        while n >= overflow:
            n /= 2
            d /= 2
            x = x >> 1

        o = (n + x) / d

        return o

    return prng


def mixkey(seed, key):
    if not seed:
        seed = time()

    for j in range(0, len(str(seed))):
        key.insert(mask & j, ord(str(seed)[j]))

    return key


In [4]:
def islist(this):
    return type(this) is list


def seedrand(func, min, max):
    return math.floor(func() * (max - min + 1)) + min


def shuffle(list, seed=None):
    if not islist(list):
        return None

    size = len(list)
    rng = seedrandom(seed)
    # print(rng)
    resp = []
    keys = []

    for i in range(0, size):
        keys.append(i)

    for i in range(0, size):
        r = seedrand(rng, 0, len(keys)-1)
        g = keys[r]
        splice(keys, r, 1)
        resp.append(list[g])

    return resp


def unshuffle(list, seed=None):
    if not islist(list):
        return None

    size = len(list)
    rng = seedrandom(seed)
    resp = []
    keys = []

    for i in range(0, size):
        resp.append(None)
        keys.append(i)

    for i in range(0, size):
        r = seedrand(rng, 0, len(keys)-1)
        g = keys[r]
        splice(keys, r, 1)
        resp[g] = list[i]

    return resp


def splice(target, start, delete_count=None, *items):
    if delete_count is None:
        delete_count = len(target) - start

    total = start + delete_count
    removed = target[start:total]
    target[start:total] = items

    return removed


In [ ]:
math.radians(10)

0.17453292519943295

In [ ]:
import random
random.random()

0.5450313438804147

In [5]:
import io
import os
import re
import math
import random
from PIL import Image

# from pycasso.shuffleseed import shuffle, unshuffle


class Canvas:
    def __init__(self, img, slice_size, seed=None, seed2=None):
        self.img = Image.open(img)
        self.slice_width, self.slice_height = slice_size
        self.seed = seed
        self.seed2 = seed2
        self.canvas = Image.new( # with original width and height of the image
            mode="RGBA",
            size=(self.img_width, self.img_height),
            color=(255, 255, 255)
        )

        if not slice_size:
            raise ValueError(
                "Invalid slice size specified: input must be "
                "greater than or equal to one."
            )

    @property
    def img_width(self):
        return self.img.size[0]

    @property
    def img_height(self):
        return self.img.size[1]

    @property
    def img_filename(self):
        return self.get_basename(self.img.filename)

    @property
    def total_parts(self):
        return math.ceil(
            self.img_width / self.slice_width
        ) * math.ceil(
            self.img_height / self.slice_height
        ) 
    def get_slices(self):
        slices = {}
        vertical_slices = math.ceil(self.img_width / self.slice_width)

        for i in range(0, self.total_parts):
            slice = {}
            row = int(i / vertical_slices)
            col = i - row * vertical_slices
            slice['x'] = col * self.slice_width
            slice['y'] = row * self.slice_height
            slice['width'] = (
                self.slice_width - (
                    0 if (
                        slice['x'] + self.slice_width <= self.img_width
                    ) else (
                        slice['x'] + self.slice_width
                    ) - self.img_width
                )
            )
            slice['height'] = (
                self.slice_height - (
                    0 if (
                        slice['y'] + self.slice_height <= self.img_height
                    ) else (
                        slice['y'] + self.slice_height
                    ) - self.img_height
                )
            )

            if f"{slice['width']}-{slice['height']}" not in slices.keys():
                slices[f"{slice['width']}-{slice['height']}"] = []
            slices[f"{slice['width']}-{slice['height']}"].append(slice)

        return slices

    def get_cols_in_group(self, slices): # what is this used for? just count #col, with corner cases
        if len(slices) == 1:
            return 1

        t = 'init'
        for i in range(0, len(slices)):
            if t == 'init':
                t = slices[i]['y']
            if t != slices[i]['y']:
                return i
                break

        return i if (self.img_height % self.slice_height) == 0 else i + 1

    def get_group(self, slices):
        this = {}
        this['slices'] = len(slices)
        this['cols'] = self.get_cols_in_group(slices)
        this['rows'] = len(slices) / this['cols']
        this['width'] = slices[0]['width'] * this['cols']
        this['height'] = slices[0]['height'] * this['rows']
        this['x'] = slices[0]['x']
        this['y'] = slices[0]['y']
        # print(this['x']) okay, just the starting pixels of the new image
        return this

    def move_rotate(self, ox, oy, px, py, rotate = True, angle = None):
        if angle is None:
          if rotate == False:
            angle = 0
          else:
            angle = rotate = random.randint(1, 360)
        x_adjustment = (random.random() - 0.5) * 2 * self.slice_width
        y_adjustment = (random.random() - 0.5) * 2 * self.slice_width
        qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
        qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
        qx = qx + x_adjustment
        qy = qy + y_adjustment
        return [qx, qy, angle, x_adjustment, y_adjustment]

    def copy_move(self, ox, oy, px, py, angle, x_adjustment, y_adjustment):
        qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
        qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
        qx = qx + x_adjustment
        qy = qy + y_adjustment
        return [qx, qy]


    @staticmethod
    def get_basename(filename):
        name, ext = os.path.splitext(os.path.basename(filename))
        return name

    @staticmethod
    def set_stdname(filename):
        pattern = re.compile(r'[?"|:<>*]', flags=re.VERBOSE)
        return pattern.sub("", str(filename))

    def set_directory(self, path):
        if not os.path.isabs(path):
            path = os.path.join(os.getcwd(), self.set_stdname(path))

        os.makedirs(os.path.dirname(path), exist_ok=True)
        return path

    def export(self, mode=None, path=None, format=None):
        try:
            if not mode:
                mode = "scramble"

            if not format:
                format = "png"

            if format == "jpg":
                format = "jpeg"

            if format == "jpeg":
                self.canvas = self.canvas.convert("RGB")

            slices = self.get_slices() # keep them

            for g in slices:
                group = self.get_group(slices[g]) # "return this"
                # print(group) # four types (sizes) of slices, each g represents each type; shuffle within each type?
                shuffle_ind = []
                for i in range(0, len(slices[g])): # seems not the ideal code
                    shuffle_ind.append(i)
                if mode == 'unscramble': # just sort shuffle_ind...
                    shuffle_ind = unshuffle(shuffle_ind, self.seed)
                if mode == 'scramble': # seems not relevant to images, just shuffle the values/numbers
                    # print(shuffle_ind)
                    shuffle_ind_1 = shuffle(shuffle_ind, self.seed) # first layer
                    shuffle_ind_2 = shuffle(shuffle_ind, self.seed2) # 2nd layer
                    # print(shuffle_ind_1, shuffle_ind_2)

                for i in range(0, len(slices[g])):
                    s = shuffle_ind_1[i]
                    row = int(s / group['cols'])
                    col = s - row * group['cols']
                    x = col * slices[g][i]['width'] # new x
                    y = row * slices[g][i]['height'] # new y
            
                    origin_x = x + slices[g][i]['width']
                    origin_y = y + slices[g][i]['height']

                    # print(x, y)
                    rx, ry, r_angle, x_adjust, y_adjust = self.move_rotate(origin_x, origin_y, x, y, rotate = True)
                    rx = int(rx)
                    ry = int(ry)
                    print(origin_x, origin_y, rx, ry)

                    

                    rx_move = int(rx - x)
                    ry_move = int(ry - y)

                    # print(rx, ry)

                    # mask = Image.new('L', front.size, 255)
                    # front = front.rotate(100, expand=True)
                    # mask = mask.rotate(100, expand=True)
                    # back.paste(front, (100,100), mask)

                    tiles = self.img.crop( # new values 
                        box=(
                            group['x'] + rx,
                            group['y'] + ry,
                            group['x'] + origin_x, 
                            group['y'] + origin_y 
                        )
                    )
                    print("1")
                    print(tiles)

                    # tiles = self.img.crop( # new values 
                    #     box=(
                    #         group['x'] + x,
                    #         group['y'] + y,
                    #         group['x'] + x + slices[g][i]['width'], # 
                    #         group['y'] + y + slices[g][i]['height'] # 
                    #     )
                    # )

                    px = slices[g][i]['x']
                    py = slices[g][i]['y']

                    rx_2 = px + rx_move 
                    ry_2 = py + ry_move

                    origin_x_2 = slices[g][i]['x'] + slices[g][i]['width']
                    origin_y_2 = slices[g][i]['y'] + slices[g][i]['height']
                    # rx_2, ry_2 = self.copy_move(origin_x_2, origin_y_2, px, py, r_angle, x_adjust, y_adjust)
                    print(origin_x_2, origin_y_2, rx_2, ry_2,)

                    

                    self.canvas.paste( # assign new values to proper slice?
                        tiles,
                        box=(
                            rx_2,
                            ry_2,
                            origin_x_2,
                            origin_y_2
                        )
                    )


                    # self.canvas.paste( # assign new values to proper slice?
                    #     tiles,
                    #     box=(
                    #         slices[g][i]['x'],
                    #         slices[g][i]['y'],
                    #         slices[g][i]['x'] + slices[g][i]['width'],
                    #         slices[g][i]['y'] + slices[g][i]['height']
                    #     )
                    # )
                    print("2")

            img_bytes = io.BytesIO()
            self.canvas.save(img_bytes, format)

            if path:
                if path.endswith('/'):
                    filename = f"pycasso-{self.img_filename}"
                    path = os.path.join(path, filename)

                output = f"{path}.{format}"

                with open(
                    self.set_directory(output)
                    if not os.path.exists(output) else output, 'wb'
                ) as file:
                    file.write(img_bytes.getvalue())

            return img_bytes

        except ValueError as error:
            raise SystemExit(f"Error: {error}")

    def close(self):
        self.img.close()


In [6]:
import io
import os
import re
import math
import random
from PIL import Image, ImageTransform

# from pycasso.shuffleseed import shuffle, unshuffle


class Canvas:
    def __init__(self, img, slice_size, seed = None, seed2 = None, seed3 = None, seed4 = None):
        self.img = Image.open(img)
        self.slice_width, self.slice_height = slice_size
        self.seed = seed
        self.seed2 = seed2
        self.seed3 = seed3
        self.seed4 = seed4
        self.canvas = Image.new( # with original width and height of the image
            mode="RGBA",
            size=(self.img_width, self.img_height),
            color=(255, 255, 255)
        )

        if not slice_size:
            raise ValueError(
                "Invalid slice size specified: input must be "
                "greater than or equal to one."
            )

    @property
    def img_width(self):
        return self.img.size[0]

    @property
    def img_height(self):
        return self.img.size[1]

    @property
    def img_filename(self):
        return self.get_basename(self.img.filename)

    @property
    def total_parts(self):
        return math.ceil(
            self.img_width / self.slice_width
        ) * math.ceil(
            self.img_height / self.slice_height
        ) 
    def get_slices(self):
        slices = {}
        vertical_slices = math.ceil(self.img_width / self.slice_width)

        for i in range(0, self.total_parts):
            slice = {}
            row = int(i / vertical_slices)
            col = i - row * vertical_slices
            slice['x'] = col * self.slice_width
            slice['y'] = row * self.slice_height
            slice['width'] = (
                self.slice_width - (
                    0 if (
                        slice['x'] + self.slice_width <= self.img_width
                    ) else (
                        slice['x'] + self.slice_width
                    ) - self.img_width
                )
            )
            slice['height'] = (
                self.slice_height - (
                    0 if (
                        slice['y'] + self.slice_height <= self.img_height
                    ) else (
                        slice['y'] + self.slice_height
                    ) - self.img_height
                )
            )

            if f"{slice['width']}-{slice['height']}" not in slices.keys():
                slices[f"{slice['width']}-{slice['height']}"] = []
            slices[f"{slice['width']}-{slice['height']}"].append(slice)

        return slices

    def get_cols_in_group(self, slices): # what is this used for? just count #col, with corner cases
        if len(slices) == 1:
            return 1

        t = 'init'
        for i in range(0, len(slices)):
            if t == 'init':
                t = slices[i]['y']
            if t != slices[i]['y']:
                return i
                break

        return i if (self.img_height % self.slice_height) == 0 else i + 1

    def get_group(self, slices):
        this = {}
        this['slices'] = len(slices)
        this['cols'] = self.get_cols_in_group(slices)
        this['rows'] = len(slices) / this['cols']
        this['width'] = slices[0]['width'] * this['cols']
        this['height'] = slices[0]['height'] * this['rows']
        this['x'] = slices[0]['x']
        this['y'] = slices[0]['y']
        # print(this['x']) okay, just the starting pixels of the new image
        return this

    def move_rotate(self, ox, oy, px, py, rotate = True, angle = None):
        if angle is None:
          if rotate == False:
            angle = 0
          else:
            angle = rotate = random.randint(1, 360)
        x_adjustment = (random.random() - 0.5) * 2 * self.slice_width
        y_adjustment = (random.random() - 0.5) * 2 * self.slice_width
        qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
        qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
        qx = qx + x_adjustment
        qy = qy + y_adjustment
        return [qx, qy, angle, x_adjustment, y_adjustment]

    def copy_move(self, ox, oy, px, py, angle, x_adjustment, y_adjustment):
        qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
        qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
        qx = qx + x_adjustment
        qy = qy + y_adjustment
        return [qx, qy]


    @staticmethod
    def get_basename(filename):
        name, ext = os.path.splitext(os.path.basename(filename))
        return name

    @staticmethod
    def set_stdname(filename):
        pattern = re.compile(r'[?"|:<>*]', flags=re.VERBOSE)
        return pattern.sub("", str(filename))

    def set_directory(self, path):
        if not os.path.isabs(path):
            path = os.path.join(os.getcwd(), self.set_stdname(path))

        os.makedirs(os.path.dirname(path), exist_ok=True)
        return path

    def export(self, mode=None, path=None, format=None):
        try:
            if not mode:
                mode = "scramble"

            if not format:
                format = "png"

            if format == "jpg":
                format = "jpeg"

            if format == "jpeg":
                self.canvas = self.canvas.convert("RGB")

            slices = self.get_slices() # keep them

            for g in slices:
                group = self.get_group(slices[g]) # "return this"
                # print(group) # four types (sizes) of slices, each g represents each type; shuffle within each type?
                shuffle_ind = []
                for i in range(0, len(slices[g])): # seems not the ideal code
                    shuffle_ind.append(i)
                if mode == 'unscramble': # just sort shuffle_ind...
                    shuffle_ind = unshuffle(shuffle_ind, self.seed)
                if mode == 'scramble': # seems not relevant to images, just shuffle the values/numbers
                    # print(shuffle_ind)
                    shuffle_ind_1 = shuffle(shuffle_ind, self.seed) # first layer
                    shuffle_ind_2 = shuffle(shuffle_ind, self.seed2) # 2nd layer
                    shuffle_order_1 = shuffle(shuffle_ind, self.seed3) # random order of drawing
                    shuffle_order_2 = shuffle(shuffle_ind, self.seed4) 
                    # print(shuffle_ind_1, shuffle_ind_2)

                for i in range(0, len(slices[g])):
                    s = shuffle_ind_1[i]
                    move_to_i = shuffle_order_1[i]
                    row = int(s / group['cols'])
                    col = s - row * group['cols']
                    x = col * slices[g][i]['width'] # new x
                    y = row * slices[g][i]['height'] # new y
            
                    # origin_x = x + slices[g][i]['width']
                    # origin_y = y + slices[g][i]['height']

                    # print(x, y)
                    # rx, ry, r_angle, x_adjust, y_adjust = self.move_rotate(origin_x, origin_y, x, y, rotate = True)
                    # rx = int(rx)
                    # ry = int(ry)
                    # print(origin_x, origin_y, rx, ry)

                    

                    # rx_move = int(rx - x)
                    # ry_move = int(ry - y)

                    # print(rx, ry)

                    # mask = Image.new('L', front.size, 255)
                    # front = front.rotate(100, expand=True)
                    # mask = mask.rotate(100, expand=True)
                    # back.paste(front, (100,100), mask)

                    # tiles = self.img.crop( # new values 
                    #     box=(
                    #         group['x'] + rx,
                    #         group['y'] + ry,
                    #         group['x'] + origin_x, 
                    #         group['y'] + origin_y 
                    #     )
                    # )
                    # print("1")
                    # print(tiles)

                    tiles = self.img.crop( # new values 
                        box=(
                            group['x'] + x,
                            group['y'] + y,
                            group['x'] + x + slices[g][i]['width'], # 
                            group['y'] + y + slices[g][i]['height'] # 
                        )
                    )

                    

                    # display(rotated_tiles)
       

                    # px = slices[g][i]['x']
                    # py = slices[g][i]['y']

                    # rx_2 = px + rx_move 
                    # ry_2 = py + ry_move

                    # origin_x_2 = slices[g][i]['x'] + slices[g][i]['width']
                    # origin_y_2 = slices[g][i]['y'] + slices[g][i]['height']
                    # rx_2, ry_2 = self.copy_move(origin_x_2, origin_y_2, px, py, r_angle, x_adjust, y_adjust)
                    # print(origin_x_2, origin_y_2, rx_2, ry_2,)

                    

                    # self.canvas.paste( # assign new values to proper slice?
                    #     tiles,
                    #     box=(
                    #         rx_2,
                    #         ry_2,
                    #         origin_x_2,
                    #         origin_y_2
                    #     )
                    # )

                    # print("1")

                    angle = random.random() * 360

                    mask = Image.new('L', (self.slice_width, self.slice_height), 255)
                    # print("2")
                    rotated_tiles = tiles.rotate(angle, expand = True)
                    # print("3")
                    mask = mask.rotate(angle, expand = True)
                    # print("4")

                    move_to_x = slices[g][move_to_i]['x'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    move_to_y = slices[g][move_to_i]['y'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    
                    # self.canvas.paste(rotated_tiles, (slices[g][i]['x'], slices[g][i]['y']), mask)
                    self.canvas.paste(rotated_tiles, (move_to_x, move_to_y), mask)
                    # print("5")


                    # self.canvas.paste( # assign new values to proper slice?
                    #     tiles,
                    #     box=(
                    #         slices[g][i]['x'],
                    #         slices[g][i]['y'],
                    #         slices[g][i]['x'] + slices[g][i]['width'],
                    #         slices[g][i]['y'] + slices[g][i]['height']
                    #     )
                    # )
                    # print("2")

            img_bytes = io.BytesIO()
            self.canvas.save(img_bytes, format)

            if path:
                if path.endswith('/'):
                    filename = f"pycasso-{self.img_filename}"
                    path = os.path.join(path, filename)

                output = f"{path}.{format}"

                with open(
                    self.set_directory(output)
                    if not os.path.exists(output) else output, 'wb'
                ) as file:
                    file.write(img_bytes.getvalue())

            return img_bytes

        except ValueError as error:
            raise SystemExit(f"Error: {error}")

    def close(self):
        self.img.close()

In [7]:
import io
import os
import re
import math
import random
from PIL import Image, ImageTransform

# from pycasso.shuffleseed import shuffle, unshuffle


class Canvas:
    def __init__(self, img, slice_size, seed = None, seed2 = None, seed3 = None, seed4 = None):
        self.img = Image.open(img)
        self.slice_width, self.slice_height = slice_size
        self.seed = seed
        self.seed2 = seed2
        self.seed3 = seed3
        self.seed4 = seed4
        self.canvas = Image.new( # with original width and height of the image
            mode="RGBA",
            size=(self.img_width, self.img_height),
            color=(255, 255, 255)
        )

        if not slice_size:
            raise ValueError(
                "Invalid slice size specified: input must be "
                "greater than or equal to one."
            )

    @property
    def img_width(self):
        return self.img.size[0]

    @property
    def img_height(self):
        return self.img.size[1]

    @property
    def img_filename(self):
        return self.get_basename(self.img.filename)

    @property
    def total_parts(self):
        return math.ceil(
            self.img_width / self.slice_width
        ) * math.ceil(
            self.img_height / self.slice_height
        ) 
    def get_slices(self):
        slices = {}
        vertical_slices = math.ceil(self.img_width / self.slice_width)

        for i in range(0, self.total_parts):
            slice = {}
            row = int(i / vertical_slices)
            col = i - row * vertical_slices
            slice['x'] = col * self.slice_width
            slice['y'] = row * self.slice_height
            slice['width'] = (
                self.slice_width - (
                    0 if (
                        slice['x'] + self.slice_width <= self.img_width
                    ) else (
                        slice['x'] + self.slice_width
                    ) - self.img_width
                )
            )
            slice['height'] = (
                self.slice_height - (
                    0 if (
                        slice['y'] + self.slice_height <= self.img_height
                    ) else (
                        slice['y'] + self.slice_height
                    ) - self.img_height
                )
            )

            if f"{slice['width']}-{slice['height']}" not in slices.keys():
                slices[f"{slice['width']}-{slice['height']}"] = []
            slices[f"{slice['width']}-{slice['height']}"].append(slice)

        return slices

    def get_cols_in_group(self, slices): # what is this used for? just count #col, with corner cases
        if len(slices) == 1:
            return 1

        t = 'init'
        for i in range(0, len(slices)):
            if t == 'init':
                t = slices[i]['y']
            if t != slices[i]['y']:
                return i
                break

        return i if (self.img_height % self.slice_height) == 0 else i + 1

    def get_group(self, slices):
        this = {}
        this['slices'] = len(slices)
        this['cols'] = self.get_cols_in_group(slices)
        this['rows'] = len(slices) / this['cols']
        this['width'] = slices[0]['width'] * this['cols']
        this['height'] = slices[0]['height'] * this['rows']
        this['x'] = slices[0]['x']
        this['y'] = slices[0]['y']
        # print(this['x']) okay, just the starting pixels of the new image
        return this

    def move_rotate(self, ox, oy, px, py, rotate = True, angle = None):
        if angle is None:
          if rotate == False:
            angle = 0
          else:
            angle = rotate = random.randint(1, 360)
        x_adjustment = (random.random() - 0.5) * 2 * self.slice_width
        y_adjustment = (random.random() - 0.5) * 2 * self.slice_width
        qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
        qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
        qx = qx + x_adjustment
        qy = qy + y_adjustment
        return [qx, qy, angle, x_adjustment, y_adjustment]

    def copy_move(self, ox, oy, px, py, angle, x_adjustment, y_adjustment):
        qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
        qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
        qx = qx + x_adjustment
        qy = qy + y_adjustment
        return [qx, qy]


    @staticmethod
    def get_basename(filename):
        name, ext = os.path.splitext(os.path.basename(filename))
        return name

    @staticmethod
    def set_stdname(filename):
        pattern = re.compile(r'[?"|:<>*]', flags=re.VERBOSE)
        return pattern.sub("", str(filename))

    def set_directory(self, path):
        if not os.path.isabs(path):
            path = os.path.join(os.getcwd(), self.set_stdname(path))

        os.makedirs(os.path.dirname(path), exist_ok=True)
        return path

    def export(self, mode=None, path=None, format=None):
        try:
            if not mode:
                mode = "scramble"

            if not format:
                format = "png"

            if format == "jpg":
                format = "jpeg"

            if format == "jpeg":
                self.canvas = self.canvas.convert("RGB")

            slices = self.get_slices() # keep them

            for g in slices:
                group = self.get_group(slices[g]) # "return this"
                # print(group) # four types (sizes) of slices, each g represents each type; shuffle within each type?
                shuffle_ind = []
                for i in range(0, len(slices[g])): # seems not the ideal code
                    shuffle_ind.append(i)
                if mode == 'unscramble': # just sort shuffle_ind...
                    shuffle_ind = unshuffle(shuffle_ind, self.seed)
                if mode == 'scramble': # seems not relevant to images, just shuffle the values/numbers
                    # print(shuffle_ind)
                    shuffle_ind_1 = shuffle(shuffle_ind, self.seed) # first layer
                    shuffle_ind_2 = shuffle(shuffle_ind, self.seed2) # 2nd layer
                    shuffle_order_1 = shuffle(shuffle_ind, self.seed3) # random order of drawing
                    shuffle_order_2 = shuffle(shuffle_ind, self.seed4) 
                    # print(shuffle_ind_1, shuffle_ind_2)

                for i in range(0, len(slices[g])):
                    s = shuffle_ind_1[i]
                    move_to_i = shuffle_order_1[i]
                    row = int(s / group['cols'])
                    col = s - row * group['cols']
                    x = col * slices[g][i]['width'] # new x
                    y = row * slices[g][i]['height'] # new y

                    tiles = self.img.crop( # new values 
                        box=(
                            group['x'] + x,
                            group['y'] + y,
                            group['x'] + x + slices[g][i]['width'], # 
                            group['y'] + y + slices[g][i]['height'] # 
                        )
                    )

                    angle = random.random() * 360
                    mask = Image.new('L', (self.slice_width, self.slice_height), 255)
                    rotated_tiles = tiles.rotate(angle, expand = True)
                    mask = mask.rotate(angle, expand = True)
                    move_to_x = slices[g][move_to_i]['x'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    move_to_y = slices[g][move_to_i]['y'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    self.canvas.paste(rotated_tiles, (move_to_x, move_to_y), mask)

                for i in range(0, len(slices[g])):
                    s = shuffle_ind_2[i]
                    move_to_i = shuffle_order_2[i]
                    row = int(s / group['cols'])
                    col = s - row * group['cols']
                    x = col * slices[g][i]['width'] # new x
                    y = row * slices[g][i]['height'] # new y

                    tiles = self.img.crop( # new values 
                        box=(
                            group['x'] + x,
                            group['y'] + y,
                            group['x'] + x + slices[g][i]['width'], # 
                            group['y'] + y + slices[g][i]['height'] # 
                        )
                    )

                    angle = random.random() * 360
                    mask = Image.new('L', (self.slice_width, self.slice_height), 255)
                    rotated_tiles = tiles.rotate(angle, expand = True)
                    mask = mask.rotate(angle, expand = True)
                    move_to_x = slices[g][move_to_i]['x'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    move_to_y = slices[g][move_to_i]['y'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    self.canvas.paste(rotated_tiles, (move_to_x, move_to_y), mask)
                
                for i in range(0, len(slices[g])):
                    s = shuffle_ind_1[i]
                    move_to_i = shuffle_order_1[i]
                    row = int(s / group['cols'])
                    col = s - row * group['cols']
                    x = col * slices[g][i]['width'] # new x
                    y = row * slices[g][i]['height'] # new y

                    tiles = self.img.crop( # new values 
                        box=(
                            group['x'] + x,
                            group['y'] + y,
                            group['x'] + x + slices[g][i]['width'], # 
                            group['y'] + y + slices[g][i]['height'] # 
                        )
                    )

                    angle = random.random() * 360
                    mask = Image.new('L', (self.slice_width, self.slice_height), 255)
                    rotated_tiles = tiles.rotate(angle, expand = True)
                    mask = mask.rotate(angle, expand = True)
                    move_to_x = slices[g][move_to_i]['x'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    move_to_y = slices[g][move_to_i]['y'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    self.canvas.paste(rotated_tiles, (move_to_x, move_to_y), mask)

                for i in range(0, len(slices[g])):
                    s = shuffle_ind_2[i]
                    move_to_i = shuffle_order_2[i]
                    row = int(s / group['cols'])
                    col = s - row * group['cols']
                    x = col * slices[g][i]['width'] # new x
                    y = row * slices[g][i]['height'] # new y

                    tiles = self.img.crop( # new values 
                        box=(
                            group['x'] + x,
                            group['y'] + y,
                            group['x'] + x + slices[g][i]['width'], # 
                            group['y'] + y + slices[g][i]['height'] # 
                        )
                    )

                    angle = random.random() * 360
                    mask = Image.new('L', (self.slice_width, self.slice_height), 255)
                    rotated_tiles = tiles.rotate(angle, expand = True)
                    mask = mask.rotate(angle, expand = True)
                    move_to_x = slices[g][move_to_i]['x'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    move_to_y = slices[g][move_to_i]['y'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    self.canvas.paste(rotated_tiles, (move_to_x, move_to_y), mask)

                for i in range(0, len(slices[g])):
                    s = shuffle_ind_1[i]
                    move_to_i = shuffle_order_1[i]
                    row = int(s / group['cols'])
                    col = s - row * group['cols']
                    x = col * slices[g][i]['width'] # new x
                    y = row * slices[g][i]['height'] # new y

                    tiles = self.img.crop( # new values 
                        box=(
                            group['x'] + x,
                            group['y'] + y,
                            group['x'] + x + slices[g][i]['width'], # 
                            group['y'] + y + slices[g][i]['height'] # 
                        )
                    )

                    angle = random.random() * 360
                    mask = Image.new('L', (self.slice_width, self.slice_height), 255)
                    rotated_tiles = tiles.rotate(angle, expand = True)
                    mask = mask.rotate(angle, expand = True)
                    move_to_x = slices[g][move_to_i]['x'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    move_to_y = slices[g][move_to_i]['y'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    self.canvas.paste(rotated_tiles, (move_to_x, move_to_y), mask)

                for i in range(0, len(slices[g])):
                    s = shuffle_ind_2[i]
                    move_to_i = shuffle_order_2[i]
                    row = int(s / group['cols'])
                    col = s - row * group['cols']
                    x = col * slices[g][i]['width'] # new x
                    y = row * slices[g][i]['height'] # new y

                    tiles = self.img.crop( # new values 
                        box=(
                            group['x'] + x,
                            group['y'] + y,
                            group['x'] + x + slices[g][i]['width'], # 
                            group['y'] + y + slices[g][i]['height'] # 
                        )
                    )

                    angle = random.random() * 360
                    mask = Image.new('L', (self.slice_width, self.slice_height), 255)
                    rotated_tiles = tiles.rotate(angle, expand = True)
                    mask = mask.rotate(angle, expand = True)
                    move_to_x = slices[g][move_to_i]['x'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    move_to_y = slices[g][move_to_i]['y'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    self.canvas.paste(rotated_tiles, (move_to_x, move_to_y), mask)

                for i in range(0, len(slices[g])):
                    s = shuffle_ind_1[i]
                    move_to_i = shuffle_order_1[i]
                    row = int(s / group['cols'])
                    col = s - row * group['cols']
                    x = col * slices[g][i]['width'] # new x
                    y = row * slices[g][i]['height'] # new y

                    tiles = self.img.crop( # new values 
                        box=(
                            group['x'] + x,
                            group['y'] + y,
                            group['x'] + x + slices[g][i]['width'], # 
                            group['y'] + y + slices[g][i]['height'] # 
                        )
                    )

                    angle = random.random() * 360
                    mask = Image.new('L', (self.slice_width, self.slice_height), 255)
                    rotated_tiles = tiles.rotate(angle, expand = True)
                    mask = mask.rotate(angle, expand = True)
                    move_to_x = slices[g][move_to_i]['x'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    move_to_y = slices[g][move_to_i]['y'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    self.canvas.paste(rotated_tiles, (move_to_x, move_to_y), mask)

                for i in range(0, len(slices[g])):
                    s = shuffle_ind_2[i]
                    move_to_i = shuffle_order_2[i]
                    row = int(s / group['cols'])
                    col = s - row * group['cols']
                    x = col * slices[g][i]['width'] # new x
                    y = row * slices[g][i]['height'] # new y

                    tiles = self.img.crop( # new values 
                        box=(
                            group['x'] + x,
                            group['y'] + y,
                            group['x'] + x + slices[g][i]['width'], # 
                            group['y'] + y + slices[g][i]['height'] # 
                        )
                    )

                    angle = random.random() * 360
                    mask = Image.new('L', (self.slice_width, self.slice_height), 255)
                    rotated_tiles = tiles.rotate(angle, expand = True)
                    mask = mask.rotate(angle, expand = True)
                    move_to_x = slices[g][move_to_i]['x'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    move_to_y = slices[g][move_to_i]['y'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    self.canvas.paste(rotated_tiles, (move_to_x, move_to_y), mask)
                
                for i in range(0, len(slices[g])):
                    s = shuffle_ind_1[i]
                    move_to_i = shuffle_order_1[i]
                    row = int(s / group['cols'])
                    col = s - row * group['cols']
                    x = col * slices[g][i]['width'] # new x
                    y = row * slices[g][i]['height'] # new y

                    tiles = self.img.crop( # new values 
                        box=(
                            group['x'] + x,
                            group['y'] + y,
                            group['x'] + x + slices[g][i]['width'], # 
                            group['y'] + y + slices[g][i]['height'] # 
                        )
                    )

                    angle = random.random() * 360
                    mask = Image.new('L', (self.slice_width, self.slice_height), 255)
                    rotated_tiles = tiles.rotate(angle, expand = True)
                    mask = mask.rotate(angle, expand = True)
                    move_to_x = slices[g][move_to_i]['x'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    move_to_y = slices[g][move_to_i]['y'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    self.canvas.paste(rotated_tiles, (move_to_x, move_to_y), mask)

                for i in range(0, len(slices[g])):
                    s = shuffle_ind_2[i]
                    move_to_i = shuffle_order_2[i]
                    row = int(s / group['cols'])
                    col = s - row * group['cols']
                    x = col * slices[g][i]['width'] # new x
                    y = row * slices[g][i]['height'] # new y

                    tiles = self.img.crop( # new values 
                        box=(
                            group['x'] + x,
                            group['y'] + y,
                            group['x'] + x + slices[g][i]['width'], # 
                            group['y'] + y + slices[g][i]['height'] # 
                        )
                    )

                    angle = random.random() * 360
                    mask = Image.new('L', (self.slice_width, self.slice_height), 255)
                    rotated_tiles = tiles.rotate(angle, expand = True)
                    mask = mask.rotate(angle, expand = True)
                    move_to_x = slices[g][move_to_i]['x'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    move_to_y = slices[g][move_to_i]['y'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    self.canvas.paste(rotated_tiles, (move_to_x, move_to_y), mask)
                
                for i in range(0, len(slices[g])):
                    s = shuffle_ind_1[i]
                    move_to_i = shuffle_order_1[i]
                    row = int(s / group['cols'])
                    col = s - row * group['cols']
                    x = col * slices[g][i]['width'] # new x
                    y = row * slices[g][i]['height'] # new y

                    tiles = self.img.crop( # new values 
                        box=(
                            group['x'] + x,
                            group['y'] + y,
                            group['x'] + x + slices[g][i]['width'], # 
                            group['y'] + y + slices[g][i]['height'] # 
                        )
                    )

                    angle = random.random() * 360
                    mask = Image.new('L', (self.slice_width, self.slice_height), 255)
                    rotated_tiles = tiles.rotate(angle, expand = True)
                    mask = mask.rotate(angle, expand = True)
                    move_to_x = slices[g][move_to_i]['x'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    move_to_y = slices[g][move_to_i]['y'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    self.canvas.paste(rotated_tiles, (move_to_x, move_to_y), mask)

                for i in range(0, len(slices[g])):
                    s = shuffle_ind_2[i]
                    move_to_i = shuffle_order_2[i]
                    row = int(s / group['cols'])
                    col = s - row * group['cols']
                    x = col * slices[g][i]['width'] # new x
                    y = row * slices[g][i]['height'] # new y

                    tiles = self.img.crop( # new values 
                        box=(
                            group['x'] + x,
                            group['y'] + y,
                            group['x'] + x + slices[g][i]['width'], # 
                            group['y'] + y + slices[g][i]['height'] # 
                        )
                    )

                    angle = random.random() * 360
                    mask = Image.new('L', (self.slice_width, self.slice_height), 255)
                    rotated_tiles = tiles.rotate(angle, expand = True)
                    mask = mask.rotate(angle, expand = True)
                    move_to_x = slices[g][move_to_i]['x'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    move_to_y = slices[g][move_to_i]['y'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    self.canvas.paste(rotated_tiles, (move_to_x, move_to_y), mask)
                
                for i in range(0, len(slices[g])):
                    s = shuffle_ind_1[i]
                    move_to_i = shuffle_order_1[i]
                    row = int(s / group['cols'])
                    col = s - row * group['cols']
                    x = col * slices[g][i]['width'] # new x
                    y = row * slices[g][i]['height'] # new y

                    tiles = self.img.crop( # new values 
                        box=(
                            group['x'] + x,
                            group['y'] + y,
                            group['x'] + x + slices[g][i]['width'], # 
                            group['y'] + y + slices[g][i]['height'] # 
                        )
                    )

                    angle = random.random() * 360
                    mask = Image.new('L', (self.slice_width, self.slice_height), 255)
                    rotated_tiles = tiles.rotate(angle, expand = True)
                    mask = mask.rotate(angle, expand = True)
                    move_to_x = slices[g][move_to_i]['x'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    move_to_y = slices[g][move_to_i]['y'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    self.canvas.paste(rotated_tiles, (move_to_x, move_to_y), mask)

                for i in range(0, len(slices[g])):
                    s = shuffle_ind_2[i]
                    move_to_i = shuffle_order_2[i]
                    row = int(s / group['cols'])
                    col = s - row * group['cols']
                    x = col * slices[g][i]['width'] # new x
                    y = row * slices[g][i]['height'] # new y

                    tiles = self.img.crop( # new values 
                        box=(
                            group['x'] + x,
                            group['y'] + y,
                            group['x'] + x + slices[g][i]['width'], # 
                            group['y'] + y + slices[g][i]['height'] # 
                        )
                    )

                    angle = random.random() * 360
                    mask = Image.new('L', (self.slice_width, self.slice_height), 255)
                    rotated_tiles = tiles.rotate(angle, expand = True)
                    mask = mask.rotate(angle, expand = True)
                    move_to_x = slices[g][move_to_i]['x'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    move_to_y = slices[g][move_to_i]['y'] + int((random.random() - 0.5) * 2 * self.slice_width)
                    self.canvas.paste(rotated_tiles, (move_to_x, move_to_y), mask)
                  
            img_bytes = io.BytesIO()
            self.canvas.save(img_bytes, format)

            if path:
                if path.endswith('/'):
                    filename = f"pycasso-{self.img_filename}"
                    path = os.path.join(path, filename)

                output = f"{path}.{format}"

                with open(
                    self.set_directory(output)
                    if not os.path.exists(output) else output, 'wb'
                ) as file:
                    file.write(img_bytes.getvalue())

            return img_bytes

        except ValueError as error:
            raise SystemExit(f"Error: {error}")

    def close(self):
        self.img.close()

In [ ]:
with Image.open("Mask_unscramble_dense.jpg","r") as im:
    im.rotate(45).show()

In [ ]:
import plotly.graph_objects as go

In [ ]:
im.show()

In [ ]:
display(im)

In [ ]:
img_scramble = Canvas('Mask_unscramble_dense.jpg', (120, 120), 'seed', 'rpac','zara','lego').export('scramble', 'image_output', 'jpeg')

In [ ]:
import io
image = Image.open(img_scramble)
image.show()
display(image)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import io
import os
import re
import math
from PIL import Image

class Canvas:
    def __init__(self, img, slice_size, seed=None):
        self.img = Image.open(img)
        self.slice_width, self.slice_height = slice_size
        self.seed = seed
        self.canvas = Image.new(
            mode="RGBA",
            size=(self.img_width, self.img_height),
            color=(255, 255, 255)
        )

        if not slice_size:
            raise ValueError(
                "Invalid slice size specified: input must be "
                "greater than or equal to one."
            )

    @property
    def img_width(self):
        return self.img.size[0]

    @property
    def img_height(self):
        return self.img.size[1]

    @property
    def img_filename(self):
        return self.get_basename(self.img.filename)

    @property
    def total_parts(self):
        return math.ceil(
            self.img_width / self.slice_width
        ) * math.ceil(
            self.img_height / self.slice_height
        )

    def get_slices(self):
        slices = {}
        vertical_slices = math.ceil(self.img_width / self.slice_width)

        for i in range(0, self.total_parts):
            slice = {}
            row = int(i / vertical_slices)
            col = i - row * vertical_slices
            slice['x'] = col * self.slice_width
            slice['y'] = row * self.slice_height
            slice['width'] = (
                self.slice_width - (
                    0 if (
                        slice['x'] + self.slice_width <= self.img_width
                    ) else (
                        slice['x'] + self.slice_width
                    ) - self.img_width
                )
            )
            slice['height'] = (
                self.slice_height - (
                    0 if (
                        slice['y'] + self.slice_height <= self.img_height
                    ) else (
                        slice['y'] + self.slice_height
                    ) - self.img_height
                )
            )

            if f"{slice['width']}-{slice['height']}" not in slices.keys():
                slices[f"{slice['width']}-{slice['height']}"] = []
            slices[f"{slice['width']}-{slice['height']}"].append(slice)

        return slices

    def get_cols_in_group(self, slices):
        if len(slices) == 1:
            return 1

        t = 'init'
        for i in range(0, len(slices)):
            if t == 'init':
                t = slices[i]['y']
            if t != slices[i]['y']:
                return i
                break

        return i if (self.img_height % self.slice_height) == 0 else i + 1

    def get_group(self, slices):
        this = {}
        this['slices'] = len(slices)
        this['cols'] = self.get_cols_in_group(slices)
        this['rows'] = len(slices) / this['cols']
        this['width'] = slices[0]['width'] * this['cols']
        this['height'] = slices[0]['height'] * this['rows']
        this['x'] = slices[0]['x']
        this['y'] = slices[0]['y']
        return this

    @staticmethod
    def get_basename(filename):
        name, ext = os.path.splitext(os.path.basename(filename))
        return name

    @staticmethod
    def set_stdname(filename):
        pattern = re.compile(r'[?"|:<>*]', flags=re.VERBOSE)
        return pattern.sub("", str(filename))

    def set_directory(self, path):
        if not os.path.isabs(path):
            path = os.path.join(os.getcwd(), self.set_stdname(path))

        os.makedirs(os.path.dirname(path), exist_ok=True)
        return path

    def export(self, mode=None, path=None, format=None):
        try:
            if not mode:
                mode = "scramble"

            if not format:
                format = "png"

            if format == "jpg":
                format = "jpeg"

            if format == "jpeg":
                self.canvas = self.canvas.convert("RGB")

            slices = self.get_slices()

            for g in slices:
                group = self.get_group(slices[g])
                shuffle_ind = []
                for i in range(0, len(slices[g])):
                    shuffle_ind.append(i)
                if mode == 'unscramble':
                    shuffle_ind = unshuffle(shuffle_ind, self.seed)
                if mode == 'scramble':
                    shuffle_ind = shuffle(shuffle_ind, self.seed)

                for i in range(0, len(slices[g])):
                    s = shuffle_ind[i]
                    row = int(s / group['cols'])
                    col = s - row * group['cols']
                    x = col * slices[g][i]['width']
                    y = row * slices[g][i]['height']

                    # print(group['x'] + x,
                    #       group['y'] + y,
                    #       group['x'] + x + slices[g][i]['width'],
                    #       group['y'] + y + slices[g][i]['height'])

                    tiles = self.img.crop(
                        box=(
                            group['x'] + x,
                            group['y'] + y,
                            group['x'] + x + slices[g][i]['width'],
                            group['y'] + y + slices[g][i]['height']
                        )
                    )

                    # print(slices[g][i]['x'],
                    #       slices[g][i]['y'],
                    #       slices[g][i]['x'] + slices[g][i]['width'],
                    #       slices[g][i]['y'] + slices[g][i]['height'])
                    self.canvas.paste(
                        tiles,
                        box=(
                            slices[g][i]['x'],
                            slices[g][i]['y'],
                            slices[g][i]['x'] + slices[g][i]['width'],
                            slices[g][i]['y'] + slices[g][i]['height']
                        )
                    )

            img_bytes = io.BytesIO()
            self.canvas.save(img_bytes, format)

            if path:
                if path.endswith('/'):
                    filename = f"pycasso-{self.img_filename}"
                    path = os.path.join(path, filename)

                output = f"{path}.{format}"

                with open(
                    self.set_directory(output)
                    if not os.path.exists(output) else output, 'wb'
                ) as file:
                    file.write(img_bytes.getvalue())

            return img_bytes

        except ValueError as error:
            raise SystemExit(f"Error: {error}")

    def close(self):
        self.img.close()

In [ ]:
img_scramble = Canvas('Mask_unscramble_dense.jpg', (30, 30), 'seed').export('scramble', 'image_output', 'jpeg')

In [ ]:
import io
image = Image.open(img_scramble)
image.show()
display(image)

Output hidden; open in https://colab.research.google.com to view.

Output hidden; open in https://colab.research.google.com to view.